In [19]:
import numpy as np

from torch.utils.data import DataLoader
from torchvision import datasets

from tqdm.notebook import tqdm

import MST

In [20]:
BATCH_SIZE = 32
NUM_WORKERS = 4

EPOCHS = 5

In [21]:
transform = np.array

train_dataset = datasets.MNIST(
    root='datasets',
    train=True,
    transform=transform,
    download=True
)

test_dataset = datasets.MNIST(
    root='datasets',
    train=False,
    transform=transform,
    download=True
)

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
)

In [22]:
class MyNet(MST.BasicModule):
    def __init__(self):
        super().__init__()

        self.l1 = MST.FullyConnectedLayer(28*28, 100)
        self.a1 = MST.Sigmoid()
        self.l2 = MST.FullyConnectedLayer(100, 50)
        self.a2 = MST.Sigmoid()
        self.l3 = MST.FullyConnectedLayer(50, 10)
        self.a3 = MST.Sigmoid()

    def forward(self, x):
        x = x.reshape(-1, 28*28)
        x = self.l1(x)
        x = self.a1(x)
        x = self.l2(x)
        x = self.a2(x)
        x = self.l3(x)
        x = self.a3(x)
        return x

    def backward(self, dOut):
        dN = self.a3.backward(dOut)
        dN = self.l3.backward(dN)
        dN = self.a2.backward(dN)
        dN = self.l2.backward(dN)
        dN = self.a1.backward(dN)
        dN = self.l1.backward(dN)

CELoss = MST.CrossEntropyLoss()
net = MyNet()
optimizer = MST.SGD(net, 0.01)

In [23]:
def train(net : MST.BasicModule, optimizer : MST.SGD, criterion : MST.BasicModule):
    running_loss = 0
    for images, labels in train_dataloader:
        labels = np.array(labels)
        
        output = net(images)

        loss = criterion(output, labels)
        net.backward(criterion.backward())
        optimizer.step()

        running_loss += loss
    train_loss = running_loss / len(train_dataloader)
    return train_loss


def valid(net : MST.BasicModule, criterion : MST.BasicModule):
    running_loss = 0
    correct_total = 0
    for images, labels in test_dataloader:
        labels = np.array(labels)

        output = net(images)

        loss = criterion(output, labels)
        running_loss += loss

        pred = np.argmax(output, axis=1, keepdims=True)
        correct_total += np.sum(pred[pred==labels.reshape(-1, 1)].astype(bool))
        
    precison = correct_total / len(test_dataloader.dataset)
    valid_loss = running_loss / len(test_dataloader)
    return valid_loss, precison

In [24]:
for epoch in (pbar := tqdm(range(EPOCHS))):
    train_loss = train(net, optimizer, CELoss)
    valid_loss, prec = valid(net, CELoss)
    print(f"[{epoch}] train/valid loss: {train_loss:.4f}/{valid_loss:.4f} prec: {prec:.4f}")
    pbar.set_description(f"train/valid loss: {train_loss:.4f}/{valid_loss:.4f} prec: {prec:.4f}")

  0%|          | 0/5 [00:00<?, ?it/s]

[0] train/valid loss: 1.5593/1.3619 prec: 0.5398
[1] train/valid loss: 1.2392/1.1621 prec: 0.6032
[2] train/valid loss: 1.0886/1.0306 prec: 0.6467
[3] train/valid loss: 0.9939/0.9375 prec: 0.6763
[4] train/valid loss: 0.9256/0.9055 prec: 0.6868
